#Modelo con TensorFlow OD API

In [1]:
#Importamos librerias Necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import os
import re
import cv2
import random
from PIL import Image, ImageFilter, ImageDraw, ImageFont
#from IPython.display import Image as ImageDisplay
#from IPython.core.display import HTML
#from IPython.display import display, Image as IPImage
#from io import BytesIO
#import base64
#from sklearn.model_selection import train_test_split
import tensorflow as tf

2025-06-21 22:37:40.072568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750556260.274201  251499 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750556260.327334  251499 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750556260.786048  251499 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750556260.786088  251499 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750556260.786090  251499 computation_placer.cc:177] computation placer alr

In [2]:
imagenes_np = np.load('/home/garzamorada/my_character_detector_workspace/ocr_font_images_dataset.npy', allow_pickle=True)
imagenes_df = pd.read_csv('/home/garzamorada/my_character_detector_workspace/ocr_font_labels_dataset.csv')
path_tfrecord = '/home/garzamorada/my_character_detector_workspace/eval/tfrecord'
os.makedirs(path_tfrecord, exist_ok=True)

In [3]:
caracteres = "ABCDEFGHIJKLMNOPQRSTUVWXYZÑabcdefghijklmnopqrstuvwxyzñáéíóúü0123456789.,;:?¿!¡-—&*/+=-_@#$%"

# Eliminar duplicados preservando orden
caracteres = ''.join(dict.fromkeys(caracteres))

label_list = [{"id": idx, "label": c} for idx, c in enumerate(caracteres, start=1)]


def get_id_by_char(label, labels=label_list):
    for item in labels:
        if item["label"] == label:
            return item["id"]
    return None

def generar_label_map(label_list, ruta_archivo):
    with open(ruta_archivo, 'w', encoding='utf-8') as f:
        for item in label_list:
            f.write("item {\n")
            f.write(f"  id: {item['id']}\n")
            f.write(f"  name: '{item['label']}'\n")
            f.write("}\n")

generar_label_map(label_list, "/home/garzamorada/my_character_detector_workspace/label_map.pbtxt")

## Generador de imagenes

In [4]:
# Generador de tfrecord
def guardar_imagen_y_tfrecord(imagen_np, bboxes, labels, chars, output_dir, nombre_base='img', idx=0):
    """
    Guarda una imagen en disco y genera un archivo TFRecord correspondiente.

    Args:
        imagen_np: np.ndarray (H, W) en escala de grises.
        bboxes: lista de bboxes en formato [xmin, ymin, xmax, ymax].
        labels: lista de strings de etiquetas correspondientes a cada bbox.
        output_dir: carpeta donde guardar los archivos.
        nombre_base: nombre base para los archivos (ej: 'img' → img_001.png).
        idx: índice numérico único para el nombre de archivo.
    """
    os.makedirs(output_dir, exist_ok=True)

    # 1. Guardar imagen en PNG
    filename = f'{nombre_base}_{idx:04d}.png'
    path_imagen = os.path.join(output_dir, filename)
    Image.fromarray(imagen_np).save(path_imagen)

    # 2. Codificar imagen
    with tf.io.gfile.GFile(path_imagen, 'rb') as fid:
        encoded_image_data = fid.read()
    height, width = imagen_np.shape

    # 3. Normalizar bboxes
    xmins = [box[0] / width for box in bboxes]
    xmaxs = [box[2] / width for box in bboxes]
    ymins = [box[1] / height for box in bboxes]
    ymaxs = [box[3] / height for box in bboxes]
    classes_text = [char.encode('utf8') for char in chars]
    classes_ids = labels

    # 4. Crear TF Example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'png'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes_ids)),
    }))

    # 5. Guardar TFRecord
    tfrecord_path = os.path.join(output_dir, f'{nombre_base}_{idx:04d}.tfrecord')
    with tf.io.TFRecordWriter(tfrecord_path) as writer:
        writer.write(tf_example.SerializeToString())

    return path_imagen, tfrecord_path

In [5]:
# --- 1. Funciones para Generar Fondos Grandes (224x224px) ---
def create_noise_background(image_shape, gray_center=128, noise_amplitude=50):
    """
    Crea un fondo de ruido aleatorio predominantemente gris, con cierta variación.

    Args:
        image_shape (tuple): Las dimensiones (altura, anchura) de la imagen.
        gray_center (int): El valor central de gris para el ruido (0-255).
        noise_amplitude (int): La amplitud del ruido alrededor del centro.

    Returns:
        np.ndarray: Un array NumPy que representa el fondo de ruido.
    """
    min_val = max(0, gray_center - noise_amplitude)
    max_val = min(255, gray_center + noise_amplitude)
    noise = np.random.randint(min_val, max_val + 1, image_shape, dtype=np.uint8)
    return noise

def create_large_background(img_size=(640, 640), bg_type="white"):
    """
    Crea un fondo grande (224x224) del tipo especificado.

    Args:
        img_size (tuple): Dimensiones de la imagen de fondo (ancho, alto).
        bg_type (str): Tipo de fondo ('white', 'black', 'gray', 'noise').

    Returns:
        np.ndarray: Imagen de fondo en escala de grises.
    """
    if bg_type == "white":
        return np.full(img_size, 255, dtype=np.uint8)
    elif bg_type == "black":
        return np.full(img_size, 0, dtype=np.uint8)
    elif bg_type == "gray":
        return np.full(img_size, 128, dtype=np.uint8)
    elif bg_type == "noise":
        return create_noise_background(img_size) # Usa la función de ruido con parámetros por defecto
    else:
        raise ValueError("Tipo de fondo no reconocido. Use 'white', 'black', 'gray', 'noise'.")

In [6]:
# --- 2. Función para Obtener Posiciones sin Solapamiento ---

def get_non_overlapping_positions(
    num_chars,
    char_size=(32, 32), # Tamaño de cada carácter de tu dataset original
    large_img_size=(640, 640), # Tamaño de la imagen de salida
    layout_type="random", # Tipos de disposición: 'random', '1_row', '2_rows', '1_column'
    padding=0 # Espacio mínimo entre caracteres
):
    """
    Calcula posiciones para caracteres sin solapamiento dentro de una imagen grande.

    Args:
        num_chars (int): Número de caracteres a posicionar.
        char_size (tuple): Dimensiones de cada carácter (ancho, alto).
        large_img_size (tuple): Dimensiones de la imagen grande (ancho, alto).
        layout_type (str): Tipo de disposición ('random', '1_row', '2_rows', '1_column').
        padding (int): Espacio mínimo en píxeles entre los caracteres y los bordes.

    Returns:
        list: Lista de tuplas (x, y) para cada posición del carácter (esquina superior izquierda).
              Retorna [] si no se pueden colocar todos los caracteres solicitados.
    """

    char_w, char_h = char_size
    img_w, img_h = large_img_size
    positions = []
    placed_bboxes = [] # Almacena (x, y, x+w, y+h) de los caracteres ya colocados, incluyendo padding

    # Función auxiliar para verificar solapamiento con padding
    def is_overlapping(new_x, new_y):
        # Bbox del nuevo caracter incluyendo el padding
        new_bbox_padded = (new_x - padding, new_y - padding,
                           new_x + char_w + padding, new_y + char_h + padding)

        for p_bbox_padded in placed_bboxes:
            # Check for overlap (AABB intersection)
            if not (new_bbox_padded[2] < p_bbox_padded[0] or
                    new_bbox_padded[0] > p_bbox_padded[2] or
                    new_bbox_padded[3] < p_bbox_padded[1] or
                    new_bbox_padded[1] > p_bbox_padded[3]):
                return True
        return False

    if num_chars == 0:
        return []

    if layout_type == "1_row":
        total_width_needed = num_chars * (char_w + padding)
        if total_width_needed > img_w:
            num_chars = int(img_w / (char_w + padding)) - 1
            total_width_needed = num_chars * (char_w + padding)

        start_x = (img_w - total_width_needed) // 2
        y = (img_h - char_h) // 2 # Centrado vertical
        for i in range(num_chars):
            x = start_x + i * (char_w + padding)
            positions.append((x, y))
            placed_bboxes.append((x, y, x + char_w, y + char_h)) # Guardamos el bbox real, is_overlapping usa su propia versión con padding

    elif layout_type == "2_rows":
        # Distribuir caracteres entre las dos filas
        chars_per_row_upper = (num_chars + 1) // 2
        chars_per_row_lower = num_chars // 2 # Opcion 1: para num_chars impar, la fila superior tiene 1 más.
                                            # Opcion 2: chars_per_row_lower = num_chars - chars_per_row_upper;

        total_width_upper = chars_per_row_upper * (char_w + padding)
        total_width_lower = chars_per_row_lower * (char_w + padding)

        if max(total_width_upper, total_width_lower) > img_w or (char_h * 2 + padding) > img_h:
            num_chars = (int(img_w / (char_w + padding)) - 1)*2
            chars_per_row_upper = (num_chars) // 2
            chars_per_row_lower = num_chars // 2 # Opcion 1: para num_chars impar, la fila superior tiene 1 más.
                                            # Opcion 2: chars_per_row_lower = num_chars - chars_per_row_upper;
            total_width_upper = chars_per_row_upper * (char_w + padding)
            total_width_lower = chars_per_row_lower * (char_w + padding)

        # Calcular alturas de las filas (distribución equitativa o fija)
        # Aquí, las dos filas se centran verticalmente dentro del espacio total.
        total_content_height = char_h * 2 + padding # Dos alturas de caracter mas un padding entre ellas
        start_y = (img_h - total_content_height) // 2

        y_upper = start_y
        y_lower = start_y + char_h + padding

        # Posicionar en fila superior
        start_x_upper = (img_w - total_width_upper) // 2
        for i in range(chars_per_row_upper):
            x = start_x_upper + i * (char_w + padding)
            positions.append((x, y_upper))
            placed_bboxes.append((x, y_upper, x + char_w, y_upper + char_h))

        # Posicionar en fila inferior
        start_x_lower = (img_w - total_width_lower) // 2
        for i in range(chars_per_row_lower):
            x = start_x_lower + i * (char_w + padding)
            positions.append((x, y_lower))
            placed_bboxes.append((x, y_lower, x + char_w, y_lower + char_h))

    elif layout_type == "1_column":
        total_height_needed = num_chars * (char_h + padding)
        if total_height_needed > img_h:
            num_chars = int(img_h / (char_h + padding)) - 1
            total_height_needed = num_chars * (char_h + padding)

        x = (img_w - char_w) // 2 # Centrado horizontal
        start_y = (img_h - total_height_needed) // 2
        for i in range(num_chars):
            y = start_y + i * (char_h + padding)
            positions.append((x, y))
            placed_bboxes.append((x, y, x + char_w, y + char_h))

    elif layout_type == "random":
        # Asegurarse de que el tamaño de la imagen grande sea divisible por el tamaño del caracter

        grid_cols = img_w // char_w
        grid_rows = img_h // char_h

        total_grid_cells = grid_cols * grid_rows

        if num_chars > total_grid_cells:
            num_chars = total_grid_cells // 2

        # Seleccionar celdas de la grilla al azar sin repetición
        selected_cell_indices = random.sample(range(total_grid_cells), num_chars)

        for cell_idx in selected_cell_indices:
            row = cell_idx // grid_cols
            col = cell_idx % grid_cols

            x = col * char_w
            y = row * char_h
            positions.append((x, y))
    else:
        raise ValueError("Tipo de disposición no reconocido. Use 'random', '1_row', '2_rows', '1_column'.")

    return positions

In [7]:
def generate_composed_dataset(
    original_numpy_dataset, original_pandas_dataset,
    output_img_count, # Número de imágenes compuestas a generar
    min_chars_per_img=0,
    max_chars_per_img=10,
    large_img_size=(640, 640),
    char_size=(64, 64), # Este es el tamaño fijo de tus caracteres originales
    background_types=["white", "black", "gray", "noise"],
    layout_types=["random", "1_row", "2_rows", "1_column"]
):
    """
    Genera un nuevo dataset de imágenes compuestas (224x224) con múltiples caracteres (32x32).

    Args:
        original_numpy_dataset (np.ndarray): Tu dataset de NumPy con caracteres individuales (32x32).
        output_img_count (int): Número de imágenes grandes a generar.
        min_chars_per_img (int): Mínimo de caracteres a insertar por imagen grande.
        max_chars_per_img (int): Máximo de caracteres a insertar por imagen grande.
        large_img_size (tuple): Dimensiones de las imágenes de salida (ancho, alto).
        char_size (tuple): Dimensiones de los caracteres individuales (ancho, alto).
        background_types (list): Tipos de fondo para las imágenes grandes.
        layout_types (list): Tipos de disposición de caracteres dentro de la imagen grande.

    Returns:
        tuple: (composed_numpy_dataset, composed_pandas_dataset)
               composed_numpy_dataset: Array NumPy con imágenes compuestas y sus bboxes adaptados.
               composed_pandas_dataset: DataFrame de Pandas con metadatos de cada imagen compuesta.
    """

    composed_numpy_data = [] # Lista para los elementos del nuevo NumPy dataset
    composed_pandas_data = [] # Lista para los elementos del nuevo Pandas DataFrame
    global_composed_index = 0

    nominal_char_w, nominal_char_h = char_size

    if original_numpy_dataset.size == 0:
        print("El dataset de caracteres original está vacío. No se pueden generar imágenes compuestas.")
        return np.array([], dtype=[('image', object), ('bboxes', object), ('index', int)]), pd.DataFrame()

    for _ in range(output_img_count):
        if ((global_composed_index + 1) * 100) % output_img_count == 0:
            print(f"realizado {((global_composed_index + 1) * 100) // output_img_count}% ")
        num_chars_to_place = random.randint(min_chars_per_img, max_chars_per_img)

        # 1. Elegir tipo de fondo y generarlo
        #bg_type = random.choice(background_types)
        bg_type = 'black'
        background_img = create_large_background(large_img_size, bg_type)
        composed_image = background_img.copy()

        # 2. Elegir tipo de disposición y obtener posiciones
        layout_type = random.choice(layout_types)
        char_positions = get_non_overlapping_positions(
            num_chars_to_place, char_size, large_img_size, layout_type
        )

        num_chars_effectively_placed = len(char_positions)
        current_image_bboxes = [] # Almacenará los bboxes adaptados para esta imagen grande

        # 3. Seleccionar caracteres del dataset original y colocarlos
        # Solo seleccionamos el número de caracteres que realmente pudimos posicionar
        selected_char_indices = random.sample(
            range(original_numpy_dataset.shape[0]), min(num_chars_effectively_placed, original_numpy_dataset.shape[0])
        )
        labels=[]
        chars=[]
        for i, pos in enumerate(char_positions):
            if i >= len(selected_char_indices): # En caso de que se hayan encontrado más posiciones que caracteres seleccionados
                break

            char_data_index = selected_char_indices[i]
            char_entry = original_numpy_dataset[char_data_index]
            char_index = char_entry['index']
            char_data = original_pandas_dataset[original_pandas_dataset['index'] == char_index].iloc[0]
            char_label = char_data['character']
            chars.append(char_label)
            id=get_id_by_char(char_label)
            labels.append(id)

            # Obtener la imagen del caracter y su bbox ORIGINAL directamente del dataset.
            char_image = char_entry['image']    
            char_image = cv2.resize(char_image, (64, 64), interpolation=cv2.INTER_AREA)
            original_char_bbox = char_entry['bbox'] # Bbox original de la imagen 32x32px

            x_offset, y_offset = pos # Posición (top-left) donde pegamos la imagen 32x32px en la 224x224px

            # Superponer el carácter: Asignación directa del ROI
            # Esto copia la imagen de 32x32px tal cual (con su propio fondo)
            # a la posición en la imagen compuesta.
            composed_image[y_offset : y_offset + nominal_char_h, x_offset : x_offset + nominal_char_w] = char_image

            # Adaptar el bbox original (de 32x32px) a la posición en la imagen grande (224x224px)
            # Sumamos el offset de la posición donde pegamos la imagen pequeña
            adapted_x_min = original_char_bbox[0] + x_offset
            adapted_y_min = original_char_bbox[1] + y_offset
            adapted_x_max = original_char_bbox[2] + x_offset
            adapted_y_max = original_char_bbox[3] + y_offset

            current_image_bboxes.append((adapted_x_min, adapted_y_min, adapted_x_max, adapted_y_max))

        # 4. Añadir a los datasets de salida
        """composed_numpy_data.append((composed_image, current_image_bboxes, global_composed_index))"""

        """composed_pandas_data.append({
            'index': global_composed_index,
            'num_chars': num_chars_effectively_placed,
            'background_type': bg_type,
            'layout_type': layout_type,
            'has_chars': bool(current_image_bboxes) # True si hay chars, False si no
        })"""
        ruta_imagen, ruta_record = guardar_imagen_y_tfrecord(composed_image, current_image_bboxes, labels, chars, path_tfrecord, nombre_base='img', idx=global_composed_index)

        global_composed_index += 1

    # Convertir las listas a arrays de NumPy y DataFrame de Pandas
    # El dtype 'object' es necesario porque las imágenes y bboxes son arrays/listas de tamaño variable
    composed_numpy_dataset = np.array(composed_numpy_data, dtype=[('image', object), ('bboxes', object), ('index', int)])
    composed_pandas_dataset = pd.DataFrame(composed_pandas_data)

    return composed_numpy_dataset, composed_pandas_dataset

In [8]:
# Parámetros para la generación de imágenes compuestas
num_composed_images_to_generate = 5000 # Define cuántas imágenes grandes quieres generar
output_large_img_size = (640, 640)
input_char_size = (64, 64) # Tamaño fijo de tus caracteres de origen
min_chars_on_img = 0
max_chars_on_img = 20
background_options = ["white", "black", "gray", "noise"]
layout_options = ["random", "1_row", "2_rows", "1_column"]
original_numpy_dataset= imagenes_np
original_pandas_dataset= imagenes_df

composed_numpy_dataset, composed_pandas_dataset = generate_composed_dataset(
    original_numpy_dataset, original_pandas_dataset,
    num_composed_images_to_generate,
    min_chars_per_img=min_chars_on_img,
    max_chars_per_img=max_chars_on_img,
    large_img_size=output_large_img_size,
    char_size=input_char_size,
    background_types=background_options,
    layout_types=layout_options
)

realizado 1% 
realizado 2% 
realizado 3% 
realizado 4% 
realizado 5% 
realizado 6% 
realizado 7% 
realizado 8% 
realizado 9% 
realizado 10% 
realizado 11% 
realizado 12% 
realizado 13% 
realizado 14% 
realizado 15% 
realizado 16% 
realizado 17% 
realizado 18% 
realizado 19% 
realizado 20% 
realizado 21% 
realizado 22% 
realizado 23% 
realizado 24% 
realizado 25% 
realizado 26% 
realizado 27% 
realizado 28% 
realizado 29% 
realizado 30% 
realizado 31% 
realizado 32% 
realizado 33% 
realizado 34% 
realizado 35% 
realizado 36% 
realizado 37% 
realizado 38% 
realizado 39% 
realizado 40% 
realizado 41% 
realizado 42% 
realizado 43% 
realizado 44% 
realizado 45% 
realizado 46% 
realizado 47% 
realizado 48% 
realizado 49% 
realizado 50% 
realizado 51% 
realizado 52% 
realizado 53% 
realizado 54% 
realizado 55% 
realizado 56% 
realizado 57% 
realizado 58% 
realizado 59% 
realizado 60% 
realizado 61% 
realizado 62% 
realizado 63% 
realizado 64% 
realizado 65% 
realizado 66% 
realizado 67% 
real

In [9]:
# Muestro 10 resultados al azar

def draw_bboxes(image_np, bboxes):
    """Dibuja las bounding boxes en una imagen NumPy."""
    img_copy = image_np.copy()
    # Asegúrate de que la imagen sea BGR si es escala de grises para dibujar en color
    if len(img_copy.shape) == 2:
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_GRAY2BGR)

    for bbox in bboxes:
        # bbox is (x_min, y_min, x_max, y_max)
        x_min, y_min, x_max, y_max = bbox
        # Draw rectangle with green color (0, 255, 0) and thickness 1
        cv2.rectangle(img_copy, (x_min, y_min), (x_max, y_max), (0, 255, 0), 1)
    return img_copy

# Elegir 10 filas al azar del composed_numpy_dataset
# Como composed_numpy_dataset es un array numpy con dtype object, no podemos usar .sample().
# Podemos obtener índices aleatorios y luego acceder a esos índices.
num_samples = 20
random_indices = np.random.choice(len(composed_numpy_dataset), num_samples, replace=False)

print(f"Mostrando {num_samples} imágenes aleatorias del dataset compuesto con sus bboxes:")

for i in random_indices:
    # Acceder a la entrada del dataset numpy por índice
    composed_entry = composed_numpy_dataset[i]

    # Obtener la imagen y la lista de bboxes
    image = composed_entry['image']
    bboxes = composed_entry['bboxes']
    entry_index = composed_entry['index'] # El índice original de la entrada

    # Opcional: Obtener metadatos del pandas_dataset usando el 'index'
    pandas_row = composed_pandas_dataset[composed_pandas_dataset['index'] == entry_index].iloc[0]
    print(f"\n--- Imagen Index: {entry_index} ---")
    print(f"Número de caracteres: {pandas_row['num_chars']}, Fondo: {pandas_row['background_type']}, Disposición: {pandas_row['layout_type']}")
    print(f"Bboxes encontrados: {len(bboxes)}")


    # Dibujar los bboxes en la imagen
    image_with_bboxes = draw_bboxes(image, bboxes)

    # Mostrar la imagen con los bboxes dibujados
    plt.imshow(image, cmap='gray')
    plt.show()



ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
print(format(label_list))
print(f"largo: {len(label_list)}")